# RAG Example Using NVIDIA API Catalog and LangChain

This notebook introduces how to use LangChain to interact with NVIDIA hosted NIM microservices like chat, embedding, and reranking models to build a simple retrieval-augmented generation (RAG) application.

## Terminology

#### RAG

- RAG is a technique for augmenting LLM knowledge with additional data.
- LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on.
- If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs.
- The process of bringing the appropriate information and inserting it into the model prompt is known as retrieval augmented generation (RAG).

The preceding summary of RAG originates in the LangChain v0.2 tutorial [Build a RAG App](https://python.langchain.com/v0.2/docs/tutorials/rag/) tutorial in the LangChain v0.2 documentation.

#### NIM

- [NIM microservices](https://developer.nvidia.com/blog/nvidia-nim-offers-optimized-inference-microservices-for-deploying-ai-models-at-scale/) are containerized microservices that simplify the deployment of generative AI models like LLMs and are optimized to run on NVIDIA GPUs.
- NIM microservices support models across domains like chat, embedding, reranking, and more from both the community and NVIDIA.

#### NVIDIA API Catalog

- [NVIDIA API Catalog](https://build.nvidia.com/explore/discover) is a hosted platform for accessing a wide range of microservices online.
- You can test models on the catalog and then export them with an NVIDIA AI Enterprise license for on-premises or cloud deployment

#### langchain-nvidia-ai-endpoints

- The [`langchain-nvidia-ai-endpoints`](https://pypi.org/project/langchain-nvidia-ai-endpoints/) Python package contains LangChain integrations for building applications that communicate with NVIDIA NIM microservices.

## Installation and Requirements

Create a Python environment (preferably with Conda) using Python version 3.10.14.
To install Jupyter Lab, refer to the [installation](https://jupyter.org/install) page.

In [3]:
# Requirements
!python3 -m pip install langchain==0.2.5
!python3 -m pip install langchain_community==0.2.5
!python3 -m pip install faiss-gpu # replace with faiss-gpu if you are using GPU
!python3 -m pip install langchain-nvidia-ai-endpoints==0.1.2
!python3 -m pip install requests faiss-cpu pdfplumber spacy beautifulsoup4 camelot-py pymupdf

  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)
  Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
  Using cached spacy-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.1 MB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached camelot_py-0.11.0-py3-none-any.whl (40 kB)
  Using cached PyMuPDF-1.24.11-cp38-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (19.6 MB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached pypdfium2-4.30.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
  Using cached cryptography-43.0.1-cp39-abi3-manylinux_2_28_x86_64.whl (4.0 MB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
  Using cached thinc-8.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.7 MB)
  Using cached jinja2-3.1.4-py3-

## Getting Started!

To get started you need an `NVIDIA_API_KEY` to use the NVIDIA API Catalog:

1) Create a free account with [NVIDIA](https://build.nvidia.com/explore/discover).
2) Click on your model of choice.
3) Under Input select the Python tab, and click **Get API Key** and then click **Generate Key**.
4) Copy and save the generated key as NVIDIA_API_KEY. From there, you should have access to the endpoints.

Available NVIDIA api keys;

1. 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
2. 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

In [26]:
import getpass
import os

nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

Enter your NVIDIA API key: ··········


## RAG Example using LLM & Embedding

### 1) Initialize the LLM

The ChatNVIDIA class is part of LangChain's integration (langchain_nvidia_ai_endpoints) with NVIDIA NIM microservices.
It allows access to NVIDIA NIM for chat applications, connecting to hosted or locally-deployed microservices.

Here we will use **mixtral-8x7b-instruct-v0.1**

In [30]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", max_tokens=1024)

# Can choose any model hosted at Nvidia API Catalog (Uncomment the below code to list the availabe models)
# ChatNVIDIA.get_available_models()

[Model(id='seallms/seallm-7b-v2.5', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-seallm-7b']),
 Model(id='upstage/solar-10.7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-solar-10_7b-instruct']),
 Model(id='writer/palmyra-med-70b-32k', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-palmyra-med-70b-32k']),
 Model(id='microsoft/phi-3-medium-4k-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-phi-3-medium-4k-instruct']),
 Model(id='google/deplot', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/google/deplot', aliases=['ai-google-deplot', 'playground_deplot', 'deplot']),
 Model(id='google/recurrentgemma-2b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-recurrentgemma-2b']),
 Model(id='google/gemma-7b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-gemma-7b', 'playground_gemma_7b', 'gemma_7b']),
 Model(id=

### 2) Intialize the embedding
NVIDIAEmbeddings is a client to NVIDIA embeddings models that provides access to a NVIDIA NIM for embedding. It can connect to a hosted NIM or a local NIM using a base URL

We selected **NV-Embed-QA** as the embedding

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="NV-Embed-QA", truncate="END")

### 3) Obtain dataset
I love taxes and work! Lets steal information about taxes and work!

We can either;

A) Walk through a webpage and find all sub-webpages and scrape the parent and children

In [ ]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import urllib.parse  # To handle URL joining
import time
from langchain_community.document_loaders import WebBaseLoader

# Max depth determines whether you wish to look into children of parents websites, else set to 0
max_depth = 2
base_domains = [
    "https://www.iras.gov.sg/taxes",
    "https://www.iras.gov.sg/schemes",
    "https://www.mom.gov.sg/passes-and-permits",
    "https://www.mom.gov.sg/employment-practices",
    "https://www.mom.gov.sg/workplace-safety-and-health"
]
scraped_data = []
visited_urls = set()

def scrape_website(base_url, max_depth, depth=0):
    """
    Recursively scrape a website by visiting links starting from base_url.

    Parameters:
    - base_url: The URL to start scraping from.
    - depth: The current recursion depth.
    - max_depth: The maximum recursion depth to avoid infinite loops.
    """
    if base_url in visited_urls or depth > max_depth:
        return

    try:
        response = requests.get(base_url)
        if response.status_code != 200:
            print(f"Failed to retrieve {base_url}")
            return
    except Exception as e:
        print(f"Error accessing {base_url}: {e}")
        return

    visited_urls.add(base_url)

    print("Current url:", base_url)
    loader = WebBaseLoader(
        web_paths=(base_url,),  # No URL fetching as we already have the HTML content
        bs_kwargs={"parse_only": SoupStrainer(['main'])},
    )
    scraped_data.append(loader.load())

    for link in BeautifulSoup(response.content, 'html.parser').find_all('a', href=True):  # Find all links on the current page
        relative_url = link['href']
        absolute_url = urllib.parse.urljoin(base_url, relative_url)
        if base_domain in absolute_url:  # Avoids external sites
            scrape_website(absolute_url, max_depth, depth + 1)

    time.sleep(0.1)  # Avoids overloading the server

for base_domain in base_domains:
    scrape_website(base_domain, max_depth)

print("Number of pages visited:", len(scraped_data))

Current url: https://www.iras.gov.sg/taxes
Current url: https://www.iras.gov.sg/taxes/individual-income-tax
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/managing-mytax-portal-account
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/understanding-my-income-tax-filing
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/tax-residency-and-tax-rates
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/what-is-taxable-what-is-not
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/tax-reliefs-rebates-and-deductions
Current url: https://www.iras.gov.sg/taxes/individual-income-tax/basics-of-individual-income-tax/receive-tax-bill-pay-tax-check-refunds
Current url: https

Current url: https://www.iras.gov.sg/taxes/international-tax/list-of-dtas-limited-dtas-and-eoi-arrangements
Current url: https://www.iras.gov.sg/taxes/withholding-tax
Current url: https://www.iras.gov.sg/taxes/withholding-tax/basics-of-withholding-tax
Current url: https://www.iras.gov.sg/taxes/withholding-tax/basics-of-withholding-tax/overview-of-withholding-tax-(WHT)
Current url: https://www.iras.gov.sg/taxes/withholding-tax/basics-of-withholding-tax/types-of-payment-and-withholding-tax-rates
Current url: https://www.iras.gov.sg/taxes/withholding-tax/payments-to-non-resident-company
Current url: https://www.iras.gov.sg/taxes/withholding-tax/payments-to-non-resident-company/payments-that-are-subject-to-withholding-tax
Current url: https://www.iras.gov.sg/taxes/withholding-tax/payments-to-non-resident-company/payments-that-are-not-subject-to-withholding-tax
Current url: https://www.iras.gov.sg/taxes/withholding-tax/payments-to-non-resident-director
Current url: https://www.iras.gov.sg/t

In [14]:
[print(data) for data in scraped_data if "https://www.iras.gov.sg/schemes/disbursement-schemes/progressive-wage-credit-scheme#title2" in data[0].metadata['source']]
# dir(scraped_data[0][0])
# print(scraped_data[0][0].metadata)

[Document(metadata={'source': 'https://www.iras.gov.sg/schemes/disbursement-schemes/progressive-wage-credit-scheme#title2'}, page_content="\n\n\n\n\n\nHome\nSchemes\nGovernment Schemes\nProgressive Wage Credit Scheme (PWCS)\n\n\n\n\n\n\n\n\n\nGovernment Schemes\n\n\n\n\nEnterprise Innovation Scheme (EIS)\n\n\nRefundable Investment Credit (RIC)\n\n\nJobs Growth Incentive (JGI)\n\n\nProgressive Wage Credit Scheme (PWCS)\nNext level\n\n\nBack\n\n\nPWCS Glossary\n\n\n\n\nUplifting Employment Credit (UEC)\n\n\nSenior Employment Credit (SEC), Enabling Employment Credit (EEC) and CPF Transition Offset (CTO)\n\n\nSelf-review for Eligibility of Payouts\n\n\nJobs Support Scheme (JSS)\nNext level\n\n\nBack\n\n\nSpecific Industries in Tiers and SSIC Codes\n\n\n\n\nWage Credit Scheme (WCS)\nNext level\n\n\nBack\n\n\nWCS Glossary\n\n\n\n\nSkillsFuture Enterprise Credit\n\n\nMediShield Life (MSHL)\n\n\n\n\n\nProgressive Wage Credit Scheme (PWCS)\n\nShare:\n\n\n\nFacebook\n\n\n\n\n\nX\n\n\n\n\n\nLinke

[None]

B) PDF extraction

In [15]:
import os
import fitz  # PyMuPDF
import requests

pdf_folder = r"./sample pdfs"
pdf_data = []

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)

    full_text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        full_text += page.get_text("text")

    return full_text

for file in os.listdir(pdf_folder):
    print("Current pdf:", file)
    pdf_path = os.path.join(pdf_folder, file)
    pdf_data.append(extract_text_from_pdf(pdf_path))

print("Number of pdfs used:", len(pdf_data))

FileNotFoundError: [Errno 2] No such file or directory: './sample pdfs'

### 4) Process the documents into vectorstore and save it to disk

Real world documents can be very long, this makes it hard to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. More on text splitting [here](https://python.langchain.com/v0.2/docs/concepts/#text-splitters)

In [17]:
# Here we create a faiss vector store from the documents and save it to disk.
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter
from tqdm import tqdm

text_splitter = CharacterTextSplitter(chunk_size=400, separator=" ", chunk_overlap=80)

documents = []

def clean_text(text):
    return text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('  ', ' ')

for docs in tqdm(scraped_data, desc='Embedding text from websites...'):  # Had to cut down data to a quarter's worth cuz no more tokens in model to run
    for i in range(len(docs)):
        documents.extend(text_splitter.split_text(clean_text(docs[i].page_content)))

for pdf in tqdm(pdf_data, desc='Embedding text from pdfs...'):  # Had to cut down data to a quarter's worth cuz no more tokens in model to run
    documents.extend(text_splitter.split_text(clean_text(pdf)))

# metadatas = []
# for i, d in enumerate(documents):
#     splits = text_splitter.split_text(d)
#     docs.extend(splits)
#     # metadatas.extend([{"source": sources[i]}] * len(splits))

# You will only need to do this once, later on we will restore the already saved vectorstore
store = FAISS.from_texts(documents, embedder)
VECTOR_STORE = './data/nv_embedding'
store.save_local(VECTOR_STORE)

Embedding text from websites...: 100%|██████████| 622/622 [00:00<00:00, 880.93it/s]
Embedding text from pdfs...: 0it [00:00, ?it/s]


To enable runtime search, we index text chunks by embedding each document split and storing these embeddings in a vector database. Later to search, we embed the query and perform a similarity search to find the stored splits with embeddings most similar to the query.

### 5) Read the previously processed & saved vectore store back

In [19]:
# Load the FAISS vectorestore back.
store = FAISS.load_local(VECTOR_STORE, embedder, allow_dangerous_deserialization=True)

### 6) Wrap the restored vectorsore into a retriever and ask our question

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

retriever = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

# Langchain's LCEL(LangChain Expression Language) Runnable protocol is used to define the chain
# LCEL allows pipe together components and functions
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Case 1: When there is no information relevant to question

In [ ]:
print(chain.invoke("What is component two hundred and twenty five million?"))

Case 2: Simple questions

In [ ]:
print(chain.invoke("How do i file taxes for my company?"))

Case 3: Complex questions

In [ ]:
print(chain.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Case 4: Realistic questions

In [32]:
print(chain.invoke("I am an employer whose firm qualifies for PWCs. From 2025 onwards, how much Co-Funding can I recieve from the government?"))
# In its current state, the model does an ethan for this quite hard question, although it at least does retrieve some (few?) relevant PWCS info.

From 2025 onwards, as an employer whose firm qualifies for PWCS (Productivity Works Credits Scheme), you can receive co-funding from the government of up to 50% for the first tier of wage increases and 15% to 30% for the second tier of wage increases. This co-funding support applies to wage increases given in qualifying year 2025 and onwards. The gross monthly wage ceiling for PWCS co-funding will be increased to $3,000 in qualifying years 2025 and 2026.

Please note that the specific rates and details of the co-funding support may be subject to changes or updates in the PWCS guidelines. It is recommended to consult the official guidelines or contact the relevant authorities for the most accurate and up-to-date information.


## RAG Example with LLM, Embedding & Reranking

In [ ]:
# Let's test a more complex query using the above LLM Embedding chain and see if the reranker can help.
chain.invoke("In which year Gustav's grandson ascended the throne?")

"The document does not provide information on when Gustav's grandson ascended the throne. There is no mention of any Gustav, his grandson, or the event of ascending the throne in the provided document."

### Enhancing accuracy for single data sources

This example demonstrates how a re-ranking model can be used to combine retrieval results and improve accuracy during retrieval of documents.

Typically, reranking is a critical piece of high-accuracy, efficient retrieval pipelines. Generally, there are two important use cases:

- Combining results from multiple data sources
- Enhancing accuracy for single data sources

Here, we focus on demonstrating only the second use case. If you want to know more, check [here](https://github.com/langchain-ai/langchain-nvidia/blob/main/libs/ai-endpoints/docs/retrievers/nvidia_rerank.ipynb)

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIARerank
from langchain_core.runnables import RunnableParallel

# We will narrow the collection to 100 results and further narrow it to 10 with the reranker.
retriever = store.as_retriever(search_kwargs={'k':100}) # typically k will be 1000 for real world use-cases
ranker = NVIDIARerank(model='nv-rerank-qa-mistral-4b:1', top_n=10)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

reranker = lambda input: ranker.compress_documents(query=input['question'], documents=input['context'])

chain_with_ranker = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | {"context": reranker, "question": lambda input: input['question']}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain_with_ranker.invoke("How do i file taxes for my company?"))

Based on the provided documents, to file taxes for your company, you need to follow these steps:

1. Ensure that you are duly authorized by your company as an 'Approver' for Corporate Tax (Filing and Applications) in Corppass. You can refer to the step-by-step guides for assistance on Corppass setup.
2. Have your Singpass and your company’s Unique Entity Number (UEN)/ Entity ID ready.
3. Visit the mytax.iras.gov.sg website to file the Corporate Income Tax Return for your company.
4. If your company is filing Form C, you need to submit the financial statements/certified accounts and tax computation(s) for the relevant Year of Assessment (YA).
5. If your company meets the qualifying conditions to file Form C-S or Form C-S (Lite), you can choose to file the simplified version, Form C-S (Lite), if your company has an annual revenue of $200,000 or below.

You can also visit the Basic Guide to Corporate Income Tax for Companies page to get help with filing your company’s tax returns for the 

In [ ]:
print(chain_with_ranker.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Based on the provided documents, if your foreign employee is injured at work, you can report the incident and claim reparations under the Work Injury Compensation Act (WICA). Specifically, the documents mention that input tax can be claimed for work injury compensation insurance that is obligatory under WICA for both local and foreign employees performing manual work or non-manual work earning $2,600 or less a month.

To report the incident and make a claim, you can visit the Ministry of Manpower (MOM) webpage on WICA or contact MOM at +65 6438 5122. However, it is important to note that medical and accident insurance premiums for your staff are generally not allowable for input tax claims under the GST (General) Regulations, unless the insurance or payment of compensation is obligatory under WICA or under any collective agreement within the meaning of the Industrial Relations Act.

Therefore, it seems that you can report the incident and claim reparations under WICA, but you should ve

#### Note:
- In this notebook, we have used NVIDIA NIM microservices from the NVIDIA API Catalog.
- The above APIs, ChatNVIDIA, NVIDIAEmbedding, and NVIDIARerank, also support self-hosted NIM microservices.
- Change the `base_url` to your deployed NIM URL.
- Example: `llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")`
- NIM can be hosted locally using Docker, following the [NVIDIA NIM for LLMs](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html) documentation.

In [ ]:
# Example Code snippet if you want to use a self-hosted NIM
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# connect to an LLM NIM running at localhost:8000, specifying a specific model
llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")